# Script to migrate Autopsy timeline to Timesketch 

In [1]:
import pandas as pd
import datetime

In [2]:
def line_prepender(filename, line):
    with open(filename, 'r+') as f:
        content = f.read()
        f.seek(0, 0)
        f.write(line.rstrip('\r\n') + '\n' + content)

Agregar el encabezado del archivo, los campos obligatorios en el caso de timesketch son:

- message String with an informative message of the event
- datetime ISO8601 format for example: 2015-07-24T19:01:01+00:00
- timestamp_desc String explaining what type of timestamp it is for example file created

The data on autopsy body file is:


- The first of the above fields is the MD5 hash of the filesystem object in question, which is not actually carried out by any of the aforementioned tools. This should be done by the investigator through the use of scripting and its usage is not required in this technical memorandum. Thirdparty software can be used to generate MD5 hashes and include md5sum, md5deep and others. It is useful to note that The Coroner’s Toolkit Bodyfile generating tool, grave-robber, actually generates MD5 hashes of filesystem objects by calling an external MD5-hashing program. Each line of output represents one specific filesystem object which is generally a file but could also be a directory, link, pipe, socket or device.

- The second field is the filesystem object’s name (e.g. C:\Windows\explorer.exe). The inode field is the specific filesystem inode or metadata address which stores the position of the filesystem object (recall that inodes are filesystem pointers).

- The mode_as_string field corresponds to the UNIX permissions of the filesystem object. If the filesystem is a non-UNIX filesystem then the permissions are mapped to UNIX permissions. The object’s type (file, directory, link, etc.) will also be specified in the mode_as_string field.

- The UID and GID fields correspond to the user and group IDs of the filesystem object, respectively. While decoding UID and GID using mactime, it is possible to specify a password file (e.g. /etc/passwd) for converting these values into real system user and group names. However, the use of a Windows SAM type file will not work for UID and GID conversion.

- The size field corresponds to the number of bytes the filesystem object has actually been allocated on the filesystem.

- The atime, mtime, ctime and crtime fields all correspond to the various MAC time filesystembased entries for the given filesystem object, each of which is stored using UNIX Epoch time .These MAC times were examined in Section 3.3. Additional information concerning various user and system-related actions and their effects on date/time metadata as well as MAC time modifications can be found in Annex A.3.

However, in order to generate a Bodyfile from the fls or ils tools, it is necessary to specify the appropriate parameters to either program. On the other hand, the mac-robber program does not require any parameters beyond a listing of which specific directories to scan in order to generate a Bodyfile.

In [3]:
line = 'MD5|name|inode|mode_as_string|UID|GID|size|atime|mtime|ctime|crtime'

# Example: 0|D:/Program Files (x86) ($FILE_NAME)|141-48-2|d/d-wx-wx-wx|0|0|104|1309045609|1309045609|1309045609|1309045609

line_prepender('body', line)

MAC times, sometimes also called MACB times, are the date/time filesystem metadata associations used to express the specific date/time metadata different filesystems use. These expressions are used by both the Bodyfile and TLN formats and are represented as m, a, c and b, respectively. More specifically, M corresponds to a filesystem unit’s23 modification time and is often denoted as m and referred to as mtime. A filesystem unit’s last recorded access time, A, as represented by a is commonly referred to as atime. The last change or modification date/time 23 A unit is a file, link, directory, pipe, device or any other type of valid filesystem object. 20 DRDC Valcartier TM 2011-216 recorded for a given filesystem unit, C, is represented as c and referred to as ctime. Finally, a filesystem unit’s creation time, B24, commonly referred to as crtime, is denoted as b.

Como fecha global utilizaremos crtime que es descrita como ntfs en el documento https://apps.dtic.mil/sti/pdfs/AD1003976.pdf

como timestamp_desc utilizaremos autopsy timeline.

como message utilizaremos name.

In [4]:
df = pd.read_csv("body", sep="|")

In [5]:
df['crtime'] = df['crtime'].apply(lambda x:datetime.datetime.fromtimestamp(x).isoformat())
df['atime'] = df['atime'].apply(lambda x:datetime.datetime.fromtimestamp(x).isoformat())
df['mtime'] = df['mtime'].apply(lambda x:datetime.datetime.fromtimestamp(x).isoformat())
df['ctime'] = df['ctime'].apply(lambda x:datetime.datetime.fromtimestamp(x).isoformat())

df['timestamp_desc'] = 'Autopsy Timeline'
df = df.rename(columns={'name': 'message', 'crtime': 'datetime', 'atime': 'access', 'mtime': 'modification', 'ctime': 'lastChange'})

In [6]:
df

,MD5,message,inode,mode_as_string,UID,GID,size,access,modification,lastChange,datetime,timestamp_desc
0,0,D:/Program Files (x86) ($FILE_NAME),141-48-2,d/d-wx-wx-wx,0,0,104,2011-06-25T18:46:49,2011-06-25T18:46:49,2011-06-25T18:46:49,2011-06-25T18:46:49,Autopsy Timeline
1,0,D:/Program Files (x86),141-144-6,d/d-wx-wx-wx,0,0,56,2022-03-10T07:30:02,2022-03-10T07:30:02,2022-03-10T07:30:02,2009-07-13T22:20:08,Autopsy Timeline
2,0,D:/Program Files (x86)/AnyDeskMSI ($FILE_NAME),266523-48-2,d/drwxrwxrwx,0,0,86,2022-01-31T00:00:53,2022-01-31T00:00:53,2022-01-31T00:00:53,2022-01-31T00:00:53,Autopsy Timeline
3,0,D:/Program Files (x86)/AnyDeskMSI,266523-144-1,d/drwxrwxrwx,0,0,272,2022-01-31T01:34:39,2022-01-31T01:34:39,2022-01-31T01:34:39,2022-01-31T00:00:53,Autopsy Timeline
4,0,D:/Program Files (x86)/AnyDeskMSI/AnyDeskMSI.e...,266491-48-2,r/rrwxrwxrwx,0,0,94,2022-01-31T00:00:53,2022-01-31T00:00:53,2022-01-31T00:00:53,2022-01-31T00:00:53,Autopsy Timeline
...,...,...,...,...,...,...,...,...,...,...,...,...
598979,0,C:/System Volume Information/tracking.log ($FI...,98-48-5,r/rr-xr-xr-x,0,0,90,2011-06-25T17:52:18,2011-06-25T17:52:18,2011-06-25T17:52:18,2011-06-25T17:52:18,Autopsy Timeline
598980,0,C:/System Volume Information/tracking.log,98-128-4,r/rr-xr-xr-x,0,0,20480,2011-06-25T17:52:18,2011-06-25T17:52:18,2011-06-25T17:52:18,2011-06-25T17:52:18,Autopsy Timeline
598981,0,C:/System Volume Information/{3808876b-c176-4e...,100-48-2,r/rr-xr-xr-x,0,0,142,2022-03-10T07:33:22,2022-03-10T07:33:22,2022-03-10T07:33:22,2022-03-10T07:33:22,Autopsy Timeline
598982,0,C:/System Volume Information/{3808876b-c176-4e...,100-128-1,r/rr-xr-xr-x,0,0,65536,2022-03-10T07:33:22,2022-03-10T07:33:22,2022-03-10T07:33:22,2022-03-10T07:33:22,Autopsy Timeline


## Timesketch to timeline

In [7]:
df.to_csv('timeline.csv')